# Prompting Engineering 
--- 
### *Prompt Engineering*: process of crafting prompt text for a given model/parameters to enable the model to append to your prompt or continue your prompt. 
- Includes instructions, context, examples, and cue. <sup>[1](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=models-prompt-tips)</sup>
    - *instruction*: imperative statement
    - *context*: include information to guide model towards desired output 
    - *examples*: indicate desired format/shape of output via examples 
    - *que*: text at the end of the prompt likely to start the generated output on a desired path 
- technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance <sup>[2](https://www.promptingguide.ai/techniques/fewshot)</sup>


## *Zero-Shot Learning*
---
### model learns how to classify classes it has not seen before <sup>[4](https://towardsdatascience.com/understanding-zero-shot-learning-making-ml-more-human-4653ac35ccab)</sup>
- give model a prompt that is a natural language instruction describing the task <sup>[7](https://arxiv.org/pdf/2005.14165.pdf)</sup>
- give model a prompt that is not part of the training data, and the model can create desired output 
    - *CAPTCHA image identification*
- machine learning models would have to be fundamentally adapted before adding new classes/changing the output criteria, while large language models do not have to be retrained <sup>[3](https://machinelearningmastery.com/what-are-zero-shot-prompting-and-few-shot-prompting/)</sup>

> ### Contrastive Language-Image Pretraining (CLIP)
> - Goal: *Classify Images Without Any Explicit Labels*
> - Two Stages: training stage and inference stage (same as supervised models)
>   - *training stage*:  CLIP learns about images via corresponding auxiliary text 
>       - auxillary text is a form of supervision via data attributes, not labels (computationally expensive)<sup>[5](https://arxiv.org/pdf/2106.02869.pdf)</sup>
>       - over time the model learns to extract more important information resulting in better output 
>   - *inference stage*: minimize difference between image encoding and corresponding text 
>       - encodings: lower-dimension representations of data, aka the most important/distinguishable information 
>       - model output = encodings of trained images   
>       - expected output = text encoding of corresponding captions 
>
> - *Contrastive Learning*: ML technique used to learn dataset features without labels <sup>[6](https://towardsdatascience.com/understanding-contrastive-learning-d5b19fd96607)</sup>
>   - model learns correlation of various data points 



## *Few-Shot Prompting*
---
### *Few-Shot Prompting*: provide examples when you are unable to properly articulate input for desired output but still want results
- no instructions necessary, the examples show the model how you expect it to respond
- works by giving *K* examples of context/completion 
    - and one final example of context where which the model is expected to provide completion 
- learning based on a broad distribution of tasks and then rapidly adapting to a new task <sup>[7](https://arxiv.org/pdf/2005.14165.pdf)</sup>
- random nature of the model = potentially different output each time <sup>[3](https://machinelearningmastery.com/what-are-zero-shot-prompting-and-few-shot-prompting/)</sup>

> Advantages: <sup>[7](https://arxiv.org/pdf/2005.14165.pdf)</sup>
> - reduction of task-specific data needed
> - reduced potential to learn a narrow distribution from a large but narrow fine-tuning dataset 
>
> Disadvantages: <sup>[7](https://arxiv.org/pdf/2005.14165.pdf)</sup>
> - results are worse than fine-tuned-models
> - small amount of task specific data still required 




# References: 
1.  [IBM Watson Docs](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=models-prompt-tips)
2.  [Prompt Guide AI](https://www.promptingguide.ai/techniques/fewshot)
3.  [MLM: What Are Zero-Shot Prompting and Few-Shot Prompting](https://machinelearningmastery.com/what-are-zero-shot-prompting-and-few-shot-prompting/)
4.  [TDS: Understanding Zero-Shot Learning Making ML More Human](https://towardsdatascience.com/understanding-zero-shot-learning-making-ml-more-human-4653ac35ccab)
5. [Integrading Auxiliary Information in Self-Supervised Learning](https://arxiv.org/pdf/2106.02869.pdf)
6. [TDS: Understanding Contrastive Learning](https://towardsdatascience.com/understanding-contrastive-learning-d5b19fd96607)
7. [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165.pdf)